In [1]:
%run to_path.py

In [2]:
from utils.path import get_git_root
from utils.load_data import LoadData
import pandas as pd
import numpy as np

data_path = get_git_root() / "data" / "raw"
data = LoadData(data_path)
data.get_data_dirs()

['bikeshare-ridership-2014-2015',
 'bikeshare-ridership-2016',
 'bikeshare-ridership-2017',
 'bikeshare-ridership-2018',
 'bikeshare-ridership-2019',
 'bikeshare-ridership-2020',
 'bikeshare-ridership-2021',
 'bikeshare-ridership-2022',
 'bikeshare-ridership-2023',
 'bikeshare-ridership-2024',
 'bikeshare-ridership-readme']

In [3]:
avoid_dirs = ['bikeshare-ridership-2014-2015', 'bikeshare-ridership-2016', 'bikeshare-ridership-readme']
data.load_datasets(omit=avoid_dirs)

Dataset 'bikeshare-ridership-2017' loaded.
Dataset 'bikeshare-ridership-2018' loaded.
Dataset 'bikeshare-ridership-2019' loaded.
Dataset 'bikeshare-ridership-2020' loaded.
Dataset 'bikeshare-ridership-2021' loaded.
Dataset 'bikeshare-ridership-2022' loaded.
Dataset 'bikeshare-ridership-2023' loaded.
Dataset 'bikeshare-ridership-2024' loaded.
All datasets loaded


In [4]:
data.list_datasets()

{'bikeshare-ridership-2017': <utils.load_data.Dataset at 0x241d42fe680>,
 'bikeshare-ridership-2018': <utils.load_data.Dataset at 0x241d201bd90>,
 'bikeshare-ridership-2019': <utils.load_data.Dataset at 0x241d21fe350>,
 'bikeshare-ridership-2020': <utils.load_data.Dataset at 0x241d201bfa0>,
 'bikeshare-ridership-2021': <utils.load_data.Dataset at 0x241d42fe380>,
 'bikeshare-ridership-2022': <utils.load_data.Dataset at 0x241d42fe770>,
 'bikeshare-ridership-2023': <utils.load_data.Dataset at 0x241d42ff040>,
 'bikeshare-ridership-2024': <utils.load_data.Dataset at 0x241d42fe5c0>}

In [5]:
for name, ds in data.list_datasets().items():
    print(name)
    # optimize so it doesnt rerun multiple loops
    ds_cols = [set(df.columns.tolist()) for df in ds.get_data_files()]
    
    union_cols = ds.get_unique_cols()
    intersection_cols = set.intersection(*ds_cols)
    difference_cols = union_cols - intersection_cols
    files_missing_col = [key for key, df in ds.list_data_files().items() if not difference_cols.issubset(df.columns)]
    
    print(f"union: {union_cols}")
    print(f"intersection: {intersection_cols}")
    print(f"difference: {difference_cols}")
    print(f"missing in: {files_missing_col}\n")

bikeshare-ridership-2017
union: {'trip_stop_time', 'from_station_name', 'from_station_id', 'user_type', 'trip_id', 'to_station_id', 'trip_duration_seconds', 'to_station_name', 'trip_start_time'}
intersection: {'trip_stop_time', 'from_station_name', 'user_type', 'trip_id', 'trip_duration_seconds', 'to_station_name', 'trip_start_time'}
difference: {'from_station_id', 'to_station_id'}
missing in: ['Bikeshare Ridership (2017 Q3).csv', 'Bikeshare Ridership (2017 Q4).csv']

bikeshare-ridership-2018
union: {'trip_stop_time', 'from_station_name', 'from_station_id', 'user_type', 'trip_id', 'to_station_id', 'trip_duration_seconds', 'to_station_name', 'trip_start_time'}
intersection: {'trip_stop_time', 'from_station_name', 'user_type', 'trip_id', 'to_station_id', 'trip_duration_seconds', 'trip_start_time', 'to_station_name', 'from_station_id'}
difference: set()
missing in: []

bikeshare-ridership-2019
union: {'Trip Id', 'Start Station Name', 'End Station Name', 'Trip  Duration', 'User Type', 'Sta

In [6]:
column_mapping = {
    'bike_id': 'Bike Id', 
    'from_station_name': 'Start Station Name', 
    'trip_stop_time': 'End Time', 
    'trip_start_time': 'Start Time', 
    'Trip  Duration': 'Trip Duration',
    'trip_duration_seconds': 'Trip Duration', 
    'model': 'Model', 
    'trip_id': 'Trip Id', 
    'from_station_id': 'Start Station Id', 
    'to_station_id': 'End Station Id', 
    'user_type': 'User Type',
    'to_station_name': 'End Station Name'}

for ds in data.get_datasets():
    for df in ds.get_data_files():
        df.rename(columns=column_mapping, inplace=True)
        
        ordered_cols = [val for key, val in column_mapping.items() if val in df.columns]
        df = df[ordered_cols]

    print(ds.get_unique_cols())

{'Trip Id', 'Start Station Name', 'End Station Name', 'User Type', 'Trip Duration', 'Start Station Id', 'End Station Id', 'Start Time', 'End Time'}
{'Trip Id', 'Start Station Name', 'End Station Name', 'User Type', 'Trip Duration', 'Start Station Id', 'End Station Id', 'Start Time', 'End Time'}
{'Trip Id', 'Start Station Name', 'End Station Name', 'User Type', 'Trip Duration', 'Start Station Id', 'End Station Id', 'Start Time', 'Bike Id', 'End Time'}
{'Trip Id', 'Start Station Name', 'End Station Name', 'User Type', 'Trip Duration', 'Start Station Id', 'End Station Id', 'Start Time', 'Bike Id', 'End Time'}
{'Trip Id', 'Start Station Name', 'End Station Name', 'User Type', 'Trip Duration', 'Start Station Id', 'End Station Id', 'Start Time', 'Bike Id', 'End Time'}
{'Trip Id', 'Start Station Name', 'End Station Name', 'User Type', 'Trip Duration', 'Start Station Id', 'End Station Id', 'Start Time', 'Bike Id', 'End Time'}
{'Trip Id', 'Start Station Name', 'End Station Name', 'User Type', '

In [7]:
# check dtype consistency
from collections import defaultdict

ds_col_dtypes = {}
for ds in data.get_datasets():    
    col_dtypes = defaultdict(set)
    for df in ds.get_data_files():
        for col, dtype in df.dtypes.items():
            col_dtypes[col].add(dtype)
    col_dtypes = dict(col_dtypes)
    
    print(f"{ds.get_name()}")
    print(col_dtypes)
    print()
    ds_col_dtypes[ds.get_name()] = col_dtypes

summary_dtypes = {k: set().union(*(col.get(k, set()) for col in ds_col_dtypes.values())) for k in {k for v in ds_col_dtypes.values() for k in v}}

print(f"Summary\n{summary_dtypes}\n")

summary_conflicts = {k: v for k, v in summary_dtypes.items() if len(summary_dtypes[k]) > 1}
print(f"Conflicting Column\n{summary_conflicts}")

bikeshare-ridership-2017
{'Trip Id': {dtype('int64')}, 'Start Time': {dtype('O')}, 'End Time': {dtype('O')}, 'Trip Duration': {dtype('int64')}, 'Start Station Id': {dtype('int64')}, 'Start Station Name': {dtype('O')}, 'End Station Id': {dtype('int64')}, 'End Station Name': {dtype('O')}, 'User Type': {dtype('O')}}

bikeshare-ridership-2018
{'Trip Id': {dtype('int64')}, 'Trip Duration': {dtype('int64')}, 'Start Station Id': {dtype('int64')}, 'Start Time': {dtype('O')}, 'Start Station Name': {dtype('O')}, 'End Time': {dtype('O')}, 'End Station Id': {dtype('int64')}, 'End Station Name': {dtype('O')}, 'User Type': {dtype('O')}}

bikeshare-ridership-2019
{'Trip Id': {dtype('int64')}, 'Trip Duration': {dtype('float64')}, 'Start Station Id': {dtype('int64')}, 'Start Time': {dtype('O')}, 'Start Station Name': {dtype('O')}, 'End Station Id': {dtype('float64'), dtype('int64')}, 'End Time': {dtype('O')}, 'End Station Name': {dtype('O')}, 'Bike Id': {dtype('int64')}, 'User Type': {dtype('O')}}

bik

In [8]:
map_conflicts = defaultdict(lambda: defaultdict(set))

for name, col_dtypes in ds_col_dtypes.items():
    for col, dtypes in col_dtypes.items():
        if col in summary_conflicts:
            for dtype in dtypes:
                if dtype in summary_conflicts[col]:
                    map_conflicts[col][dtype].add(name)
map_conflicts = dict(map_conflicts)
map_conflicts

{'Trip Duration': defaultdict(set,
             {dtype('int64'): {'bikeshare-ridership-2017',
               'bikeshare-ridership-2018',
               'bikeshare-ridership-2020',
               'bikeshare-ridership-2021',
               'bikeshare-ridership-2022',
               'bikeshare-ridership-2023',
               'bikeshare-ridership-2024'},
              dtype('float64'): {'bikeshare-ridership-2019'}}),
 'Start Station Id': defaultdict(set,
             {dtype('int64'): {'bikeshare-ridership-2017',
               'bikeshare-ridership-2018',
               'bikeshare-ridership-2019',
               'bikeshare-ridership-2020',
               'bikeshare-ridership-2021',
               'bikeshare-ridership-2022',
               'bikeshare-ridership-2023',
               'bikeshare-ridership-2024'},
              dtype('O'): {'bikeshare-ridership-2020'}}),
 'End Station Id': defaultdict(set,
             {dtype('int64'): {'bikeshare-ridership-2017',
               'bikeshare-rider

In [10]:
# check for missing values, we want to do this while refitting into schema
dat2017, dat2018, dat2019, dat2020, dat2021, dat2022, dat2023, dat2024 = data.unpack()

print("Trip Duration")
for fname, df in dat2019.list_data_files().items():
    print(f"\n{fname}\nNum Missing Values: {df['Trip Duration'].isna().sum()}")
    print(df[df['Trip Duration'].isna()]['Trip Duration'].head())

Trip Duration

2019-Q1.csv
Num Missing Values: 7
59771    NaN
74933    NaN
78219    NaN
86634    NaN
120157   NaN
Name: Trip Duration, dtype: float64

2019-Q2.csv
Num Missing Values: 4
123305   NaN
206892   NaN
218836   NaN
303891   NaN
Name: Trip Duration, dtype: float64

2019-Q3.csv
Num Missing Values: 3
449577   NaN
834139   NaN
908535   NaN
Name: Trip Duration, dtype: float64

2019-Q4.csv
Num Missing Values: 2
227766   NaN
444309   NaN
Name: Trip Duration, dtype: float64


In [11]:
for fname, df in dat2019.list_data_files().items():
    df['Trip Duration'] = df['Trip Duration'].fillna(0)
    print(f"{fname}\nNum Missing Values: {df['Trip Duration'].isna().sum()}")
    print(f"Can convert all: {(df['Trip Duration'] % 1 == 0).all()}\n")

2019-Q1.csv
Num Missing Values: 0
Can convert all: True

2019-Q2.csv
Num Missing Values: 0
Can convert all: True

2019-Q3.csv
Num Missing Values: 0
Can convert all: True

2019-Q4.csv
Num Missing Values: 0
Can convert all: True



In [12]:
for df in dat2019.get_data_files():
    df['Trip Duration'] = df['Trip Duration'].astype('int64')
    print(f"Trip Duration: {df['Trip Duration'].dtype}")

Trip Duration: int64
Trip Duration: int64
Trip Duration: int64
Trip Duration: int64


In [13]:
print("Start Station Id dtype")
for fname, df in dat2020.list_data_files().items():
    print(f"{fname}: {df['Start Station Id'].dtype}")

Start Station Id dtype
2020-01.csv: int64
2020-02.csv: int64
2020-03.csv: int64
2020-04.csv: int64
2020-05.csv: int64
2020-06.csv: int64
2020-07.csv: int64
2020-08.csv: int64
2020-09.csv: int64
2020-10.csv: object
2020-11.csv: int64
2020-12.csv: int64


In [14]:
df_2020_10 = dat2020.get_data_files("2020-10.csv")
df_2020_10.dtypes

Trip Id                int64
Trip Duration          int64
Start Station Id      object
Start Time            object
Start Station Name    object
End Station Id        object
End Time              object
End Station Name      object
Bike Id               object
User Type             object
dtype: object

In [16]:
df_2020_10 = dat2020.get_data_files("2020-10.csv")
df_2020_10_copy = df_2020_10[['Start Station Id']].copy()
df_2020_10_copy['Numeric'] = pd.to_numeric(df_2020_10_copy['Start Station Id'], errors='coerce')

anomaly = df_2020_10_copy[df_2020_10_copy['Numeric'].isna()]
anomaly[['Start Station Id']]

,Start Station Id
25640,10/03/2020 13:28
25837,10/03/2020 13:38
26029,10/03/2020 13:48
26248,10/03/2020 13:58
26560,10/03/2020 14:12
...,...
266879,10/30/2020 17:01
268551,10/30/2020 19:42
275974,10/31/2020 18:00
277412,10/31/2020 21:27


In [17]:
df_2020_10.head()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
0,9970495,918,7418,10/01/2020 00:00,College Park - Yonge St Entrance,7007,10/01/2020 00:15,College St / Huron St,5677,Annual Member
1,9970496,662,7061,10/01/2020 00:00,Dalton Rd / Bloor St W,7143,10/01/2020 00:11,Kendal Ave / Bernard Ave,6634,Annual Member
2,9970497,525,7051,10/01/2020 00:01,Wellesley St E / Yonge St (Green P),7009,10/01/2020 00:09,King St E / Jarvis St,4560,Annual Member
3,9970498,382,7004,10/01/2020 00:01,University Ave / Elm St,7050,10/01/2020 00:07,Richmond St E / Jarvis St Green P,4948,Annual Member
4,9970499,417,7051,10/01/2020 00:01,Wellesley St E / Yonge St (Green P),7292,10/01/2020 00:08,Granby St / Church St - SMART,5400,Annual Member


In [18]:
anomaly_subset = df_2020_10.loc[anomaly.index]
anomaly_subset

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
25640,10000084625,7120,10/03/2020 13:28,Gerrard St E / River St,7120,10/03/2020 13:38,Gerrard St E / River St,5250,Annual Member,NaN
25837,10000306555,7120,10/03/2020 13:38,Gerrard St E / River St,7576,10/03/2020 13:48,Front St E / Bayview Avenue,5250,Annual Member,NaN
26029,10000519608,7576,10/03/2020 13:48,Front St E / Bayview Avenue,7357,10/03/2020 13:58,Lake Shore Blvd E / Leslie St,5250,Annual Member,NaN
26248,10000755851,7357,10/03/2020 13:58,Lake Shore Blvd E / Leslie St,7313,10/03/2020 14:12,Coxwell Ave / Lake Shore Blvd E,5250,Annual Member,NaN
26560,10001076784,7313,10/03/2020 14:12,Coxwell Ave / Lake Shore Blvd E,7317,10/03/2020 14:26,Hubbard Blvd / Balsam Av,5250,Annual Member,NaN
...,...,...,...,...,...,...,...,...,...,...
266879,10281521317,7417,10/30/2020 17:01,King St W / Jordan St,7253,10/30/2020 17:06,John St / Mercer St - SMART,4888,Annual Member,NaN
268551,10283383488,7017,10/30/2020 19:42,Widmer St / Adelaide St W,7417,10/30/2020 19:50,King St W / Jordan St,5835,Annual Member,NaN
275974,10291748450,7417,10/31/2020 18:00,King St W / Jordan St,7474,10/31/2020 18:07,Clarence Square,3814,Annual Member,NaN
277412,10293410950,7474,10/31/2020 21:27,Clarence Square,7015,10/31/2020 21:42,King St W / Bay St (West Side),3771,Annual Member,NaN


In [19]:
# double check consistency
anomaly_subset['User Type'].isnull().sum() == df_2020_10['User Type'].isnull().sum()

True

In [20]:
# fill with 0 instead of NaN to maintain column type int64, we will be mapping to these indexes directly
df_2020_10.iloc[anomaly.index, 1:] = df_2020_10.iloc[anomaly.index, 1:].shift(periods=1, fill_value=0, axis='columns')
df_2020_10.iloc[anomaly.index]

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
25640,10000084625,0,7120,10/03/2020 13:28,Gerrard St E / River St,7120,10/03/2020 13:38,Gerrard St E / River St,5250,Annual Member
25837,10000306555,0,7120,10/03/2020 13:38,Gerrard St E / River St,7576,10/03/2020 13:48,Front St E / Bayview Avenue,5250,Annual Member
26029,10000519608,0,7576,10/03/2020 13:48,Front St E / Bayview Avenue,7357,10/03/2020 13:58,Lake Shore Blvd E / Leslie St,5250,Annual Member
26248,10000755851,0,7357,10/03/2020 13:58,Lake Shore Blvd E / Leslie St,7313,10/03/2020 14:12,Coxwell Ave / Lake Shore Blvd E,5250,Annual Member
26560,10001076784,0,7313,10/03/2020 14:12,Coxwell Ave / Lake Shore Blvd E,7317,10/03/2020 14:26,Hubbard Blvd / Balsam Av,5250,Annual Member
...,...,...,...,...,...,...,...,...,...,...
266879,10281521317,0,7417,10/30/2020 17:01,King St W / Jordan St,7253,10/30/2020 17:06,John St / Mercer St - SMART,4888,Annual Member
268551,10283383488,0,7017,10/30/2020 19:42,Widmer St / Adelaide St W,7417,10/30/2020 19:50,King St W / Jordan St,5835,Annual Member
275974,10291748450,0,7417,10/31/2020 18:00,King St W / Jordan St,7474,10/31/2020 18:07,Clarence Square,3814,Annual Member
277412,10293410950,0,7474,10/31/2020 21:27,Clarence Square,7015,10/31/2020 21:42,King St W / Bay St (West Side),3771,Annual Member


In [21]:
start_time = pd.to_datetime(df_2020_10.loc[anomaly.index, 'Start Time'])
end_time = pd.to_datetime(df_2020_10.loc[anomaly.index, 'End Time'])
trip_dur = (end_time - start_time).dt.total_seconds().astype('int64')

df_2020_10.loc[trip_dur.index, 'Trip Duration'] = trip_dur
df_2020_10.loc[trip_dur.index]

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
25640,10000084625,600,7120,10/03/2020 13:28,Gerrard St E / River St,7120,10/03/2020 13:38,Gerrard St E / River St,5250,Annual Member
25837,10000306555,600,7120,10/03/2020 13:38,Gerrard St E / River St,7576,10/03/2020 13:48,Front St E / Bayview Avenue,5250,Annual Member
26029,10000519608,600,7576,10/03/2020 13:48,Front St E / Bayview Avenue,7357,10/03/2020 13:58,Lake Shore Blvd E / Leslie St,5250,Annual Member
26248,10000755851,840,7357,10/03/2020 13:58,Lake Shore Blvd E / Leslie St,7313,10/03/2020 14:12,Coxwell Ave / Lake Shore Blvd E,5250,Annual Member
26560,10001076784,840,7313,10/03/2020 14:12,Coxwell Ave / Lake Shore Blvd E,7317,10/03/2020 14:26,Hubbard Blvd / Balsam Av,5250,Annual Member
...,...,...,...,...,...,...,...,...,...,...
266879,10281521317,300,7417,10/30/2020 17:01,King St W / Jordan St,7253,10/30/2020 17:06,John St / Mercer St - SMART,4888,Annual Member
268551,10283383488,480,7017,10/30/2020 19:42,Widmer St / Adelaide St W,7417,10/30/2020 19:50,King St W / Jordan St,5835,Annual Member
275974,10291748450,420,7417,10/31/2020 18:00,King St W / Jordan St,7474,10/31/2020 18:07,Clarence Square,3814,Annual Member
277412,10293410950,900,7474,10/31/2020 21:27,Clarence Square,7015,10/31/2020 21:42,King St W / Bay St (West Side),3771,Annual Member


In [22]:
df_2020_10.dtypes

Trip Id                int64
Trip Duration          int64
Start Station Id      object
Start Time            object
Start Station Name    object
End Station Id        object
End Time              object
End Station Name      object
Bike Id               object
User Type             object
dtype: object

In [23]:
df_2020_10.isnull().sum()

Trip Id                 0
Trip Duration           0
Start Station Id        0
Start Time              0
Start Station Name    164
End Station Id         60
End Time                0
End Station Name      202
Bike Id                 0
User Type               0
dtype: int64

In [35]:
df_2020_10[df_2020_10['End Station Id'].isnull()].head()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
18189,9991564,0,7235,10/02/2020 17:41,Bay St / College St (West Side) - SMART,NaN,10/02/2020 17:41,NaN,6677,Annual Member
18629,9992026,0,7129,10/02/2020 18:15,Davenport Rd / Avenue Rd,NaN,10/02/2020 18:15,NaN,1031,Annual Member
23032,9997148,182515,7168,10/03/2020 10:30,Queens Quay / Yonge St,NaN,10/05/2020 13:12,NaN,4635,Annual Member
23614,9997816,0,7391,10/03/2020 11:15,Yonge St / Dundas Sq,NaN,10/03/2020 11:15,NaN,2977,Annual Member
26512,10001030,0,7056,10/03/2020 14:10,Parliament St / Gerrard St,NaN,10/03/2020 14:10,NaN,6288,Annual Member


In [37]:
df_2020_10['Trip Duration'].loc[df_2020_10['End Station Id'].isnull()].value_counts()

Trip Duration
0         54
182515     1
587862     1
775107     1
686918     1
273235     1
252375     1
Name: count, dtype: int64

In [43]:
df_2020_10[['End Station Id', 'End Station Name']].loc[df_2020_10['End Station Id'].isnull()].isnull().sum()

End Station Id      60
End Station Name    60
dtype: int64

In [46]:
df_2020_10.loc[df_2020_10['Start Station Id'] == df_2020_10['End Station Id']].shape[0]

12017

In [47]:
df_2020_10.loc[df_2020_10['Trip Duration'] == 0].shape[0]

133

In [32]:
# adjust dtype of Start Station Id, End Station Id, Bike Id
# datetimes will be adjusted later with all data files


Identify inconsistent column names across files within each year.

Detect presence of non-standard headers, placeholder rows, or encoding issues.

Check if date/time ranges are coherent (e.g., all monthly files span the full year

Column renaming/standardization

Missing column insertion

Type enforcement

Datetime parsing

Concate all df per year

Validate schema consistency

Save